# Metoda promenljivih okolina

Metoda promenljivih okolina (eng. *variable neighbourhood search*, skraćeno VNS) je uopštenje lokalne pretrage u kojoj su definisane okoline $U_l(x)$, $1 \le l \le l_{max}$, po kojima će se sistematski vršiti pretraživanje. Metoda promenljivih okolina se bazira na sledećim principima:

* Lokalni minimum za jedan tip okoline ne mora nužno i da bude lokalni minimum za drugi tip okoline.
* Globalni minimum predstavlja lokalni minimum za sve tipove okolina.
* Za mnoge probleme u praksi, lokalni minimumi više tipova okolina su relativno blizu jedan drugog.

<img src='assets/VNS.png' style='width:600px;'>

U literaturi postoji nekoliko varijanti osnovnog algoritma među kojima se najčešće sreću redukovana, osnovna i uopštena metoda promenljivih okolina. U svim slučajevima se, ukoliko se ne pronađe bolje rešenje, prelazi u narednu okolinu. U suprotnom, ukoliko se pronađe bolje rešenje, algoritam se resetuje na prvi razmatrani tip okoline.

Kod redukovane metode promenljivih okolina (eng. *reduced variable neighbourhood search*, skraćeno RVNS) izostavljena je lokalna pretraga i umesto lokalnog minimuma bira se proizvoljno rešenje iz odgovarajuće okoline od trenutnog u cilju traženja poboljšanja. Kod osnovne metode promenljivih okolina (eng. *basic variable neighbourhood search*, skraćeno BVNS) se slučajno bira početno rešenje $x$ u tekućoj okolini, na koje se zatim primenjuje neki tip lokalne pretrage. Kod uopštene metode promenljivih okolina (eng. *general variable neighbourhood search*, skraćeno GVNS) se koriste dva nivoa lokalne pretrage, pri čemu se tada mogu upotrebiti dva tipa okolina, jedan za spoljašnju, a drugi za unutrašnju varijantu lokalne pretrage.

### Pseudokod RVNS se može prikazati na sledeći način:

* Odabrati tipove okolina $U_l(x)$, $1 \le l \le l_{max}$
* Generisati početno rešenje $x$ i postaviti vrednost najboljeg rešenja $f^∗ = f(x)$
* Dok nije zadovoljen kriterijum zaustavljanja, ponavljati sledeće korake:
    * $l = 1$
    * Dok je $l \le l_{max}$, ponavljati sledeće korake:
        * Izabrati proizvoljno rešenje $x'$ u okolini $U_l(x)$
        * Ako rešenje $x'$ ima manju vrednost funkcije cilja od rešenja $x$, onda dodeliti $x = x'$ i $l = 1$, a inače $l = l + 1$
    * Po potrebi, ažurirati vrednost $f^*$
* Ispisati vrednost $f^*$

### Pseudokod BVNS se može prikazati na sledeći način:

* Odabrati tipove okolina $U_l(x)$, $1 \le l \le l_{max}$
* Generisati početno rešenje $x$ i postaviti vrednost najboljeg rešenja $f^∗ = f(x)$
* Dok nije zadovoljen kriterijum zaustavljanja, ponavljati sledeće korake:
    * $l = 1$
    * Dok je $l \le l_{max}$, ponavljati sledeće korake:
        * Izabrati proizvoljno rešenje $x'$ u okolini $U_l(x)$
        * Primenom nekog tipa lokalne pretrage na početno rešenje $x'$ naći rešenje $x''$ sa najmanjom vrednošću funkcije cilja
        * Ako rešenje $x''$ ima manju vrednost funkcije cilja od rešenja $x$, onda dodeliti $x = x''$ i $l = 1$, a inače $l = l + 1$
    * Po potrebi, ažurirati vrednost $f^*$
* Ispisati vrednost $f^*$

### Pseudokod GVNS se može prikazati na sledeći način:

* Odabrati tipove okolina $U_l(x)$, $1 \le l \le l_{max}$ za fazu razmrdavanja
* Odabrati tipove okolina $U'_l(x)$, $1 \le l \le l'_{max}$ za fazu lokalne pretrage
* Generisati početno rešenje $x$, poboljšati ga korišćenjem RVNS i postaviti vrednost najboljeg rešenja $f^∗ = f(x)$
* Dok nije zadovoljen kriterijum zaustavljanja, ponavljati sledeće korake:
    * $l = 1$
    * Dok je $l \le l_{max}$, ponavljati sledeće korake:
        * Izabrati proizvoljno rešenje $x'$ u okolini $U_l(x)$
        * $l' = 1$
        * Dok je $l' \le l'_{max}$, ponavljati sledeće korake:
            * Naći rešenje $x''$ u okolini $U'_{l'}(x')$ sa najmanjom vrednošću funkcije cilja
            * Ako rešenje $x''$ ima manju vrednost funkcije cilja od rešenja $x'$, onda dodeliti $x' = x''$ i $l = 1$, a inače $l = l + 1$
        * Ako rešenje $x''$ ima manju vrednost funkcije cilja od rešenja $x$, onda dodeliti $x = x''$ i $l = 1$, a inače $l = l + 1$
    * Po potrebi, ažurirati vrednost $f^*$
* Ispisati vrednost $f^*$

## Primena RVNS na UFLP

Podsetimo se, ulazni podaci za prost lokacijski problem su skup korisnika $I$, skup potencijalnih lokacija za resurse $J$, cene dodeljivanja korisnika resursima $c_{ij}$ i cene uspostavljanja resursa $f_j$. Potrebno je minimizovati ukupnu cenu dodeljivanja korisnika resursima i cena upostavljanja resursa, tako da svaki korisnik bude dodeljen tačno jednom resursu. Pritom neki resursi mogu biti neiskorišćeni, a neki iskorišćeni od strane jednog ili više korisnika.

Implementacija RVNS za UFLP se neznatno razlikuje od implementacije lokalne pretrage za UFLP, s tim što su izmenjene ili dodate sledeće funkcije:

* `restore` $-$ kod prethodne verzije ove funkcije vršilo se invertovanje samo jednog elementa, a kako sada može da se desi da je više elemenata invertovano, vrši se restauracija svih onih čiji su indeksi u nizu `inverted`.

* `get_neighbor` $-$ ovde se, u zavisnosti od toga koja okolina je u pitanju, vrši invertovanje nekoliko elemenata. Okolina $U_k$ rešenja $x$ podrazumeva da je $k$ resursa invertovano. 

* `reduced_VNS` $-$ ovde je, u duhu samog algoritma, implementirana RVNS (analogno funkciji lokalne pretrage), koja će se kasnije pozvati u glavnom programu.

In [1]:
import random

Funkcija `read_input` učitava ulazne podatke. Pretpostavka je da će se u prvom redu fajla sa podacima naći  broj korisnika `number_of_users` i broj resursa `number_of_resources`, u narednih `number_of_users` redova `number_of_resources` brojeva koji zajedno predstavljaju matricu cena dodeljivanja korisnika resursima koju ćemo zvati `cost`. Niz cena uspostavljanja resursa `fixed_cost` će se učitavati iz poslednjeg reda fajla sa podacima.

In [2]:
def read_input(filename):
    input = open(filename, 'r')
    number_of_users, number_of_resources = [int(i) for i in input.readline().split()]
    cost = [[int(j) for j in input.readline().split()] for i in range(number_of_users)] 
    fixed_cost = [int(j) for j in input.readline().split()]
    return number_of_users, number_of_resources, cost, fixed_cost

In [3]:
number_of_users, number_of_resources, cost, fixed_cost = read_input('data/uflp_input.txt')

In [4]:
number_of_users

3

In [5]:
number_of_resources

3

In [6]:
cost

[[1, 12, 3], [2, 7, 41], [19, 21, 7]]

In [7]:
fixed_cost

[12, 11, 13]

Funkcijom `is_feasible` se proverava da li je rešenje dopustivo. Rešenje je predstavljeno nizom `solution` koje se sastoji od `number_of_resources` logičkih vrednosti gde vrednost `True` na $j$-tom mestu označava da je resurs uspostavljen, a `False` da nije. Rešenje je dopustivo ukoliko niz `solution` ima bar jednu vrednost `True`.

In [8]:
def is_feasible(number_of_resources, solution):
    for j in range(number_of_resources):
        if solution[j]:
            return True
    return False

Funkcijom `initialize` se generiše proizvoljno početno rešenje. Vrednost `True` će biti izabrana sa verovatnoćom `probability` koja će kasnije biti postavljena na 0.25 budući da se najčešće u optimalnom rešenju nalazi osetno manje uspostavljenih resursa nego neuspostavljenih.

In [9]:
def initialize(number_of_resources, probability):
    solution = []
    
    # u niz se dodaje True ili False vrednost sa verovatnocom probability
    for j in range(number_of_resources):
        solution.append(random.random() < probability)
        
    # ukoliko u nizu nema ni jedne True vrednosti, na proizvoljnu poziciju se upisuje vrednost True
    if not is_feasible(number_of_resources, solution):
        solution[random.randrange(number_of_resources)] = True
        
    return solution

Funkcijom `solution_value` određuje se vrednost trenutnog rešenja. Za svakog korisnika $i$ se bira resurs $j$ za koji je cena dodeljivanja najmanja. Zatim se taj resurs označava kao zauzet. O zauzetosti resursa se vodi računa u nizu `used` čije su vrednosti na početku postavljene na `False`. Na kraju se i sve vrednosti cena uspostavljanja zauzetih resursa dodaju na povratnu vrednost funkcije. Time je u njoj sadržana ukupna cena dodeljivanja i uspostavljanja za dato fiksno rešenje. Poslednje, niz `solution` se ažurira na osnovu vrednosti niza `used` budući da mogu da postoje u rešenju uspostavljeni resursi koji nisu iskorišćeni.

In [10]:
def solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution):
    value = 0.0
    used = [False] * number_of_resources
    
    for i in range(number_of_users):
        min_cost = float('inf')
        j_used = -1
        for j in range(number_of_resources):
            if solution[j] and cost[i][j] < min_cost:
                min_cost = cost[i][j]
                j_used = j
        value += min_cost
        used[j_used] = True
        
    for j in range(number_of_resources):
        if used[j]:
            value += fixed_cost[j]
            
    solution = used
    
    return value

Funkcijom `restore` se invertuje vrednost svakog indeksa rešenja $j$ koji se nalazi u nizu `inverted`. Funkcija će kasnije biti iskorišćena u algoritmu pretrage za restauiranje vrednosti rešenja ukoliko nije pronađena bolja vrednost.

In [11]:
def restore(inverted, solution):
    for j in inverted:
        solution[j] = not solution[j]

Funkcijom `get_neighbor` se dobija k-okolina rešenja `solution`. To je, podsetimo se, vrednost u kojoj je `k` resursa invertovano. 

In [12]:
def get_neighbor(k, number_of_resources, solution):
    inverted = []
    
    # invertuje se tacno k resursa
    for i in range(k):
        j = random.randrange(number_of_resources)
        while j in inverted:
            j = random.randrange(number_of_resources)
        inverted.append(j)
        solution[j] = not solution[j]
        
    # proverava se zadovoljivost ovako dobijenog resenja
    if is_feasible(number_of_resources, solution):
        return True, inverted
    else:
        restore(inverted, solution)
        return False, inverted

Ova funkcija je implemetacija u uvodu opisanog VNS algoritma. 

In [13]:
def reduced_VNS(number_of_users, number_of_resources, cost, fixed_cost, solution, max_iters, k_max):
    
    curr_value = solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution)
    best_value = curr_value
    best_solution = solution.copy()
    
    # sve dok se ne dostigne maksimalni broj iteracija algoritma
    for i in range(max_iters):
        
        # ispituju se okoline resenja 
        k = 0
        while k < k_max:
            
            # trazi se resenje u k-okolini koje je zadovoljivo
            feasible, inverted = get_neighbor(k, number_of_resources, solution)
            while not feasible:    
                feasible, inverted = get_neighbor(k, number_of_resources, solution)
                
            # zatim se izracunava vrednost novog resenja
            new_value = solution_value(number_of_users, number_of_resources, cost, fixed_cost, solution)
            
             # ukoliko je nova vrednost bolja od najbolje, vrsimo azuriranje
            if new_value < best_value:
                best_value = new_value
                best_solution = solution.copy()
            
            # ukoliko je nova vrednost manja od tekuce vrsi se azuriranje vrednosti i ceo proces se resetuje 
            if new_value < curr_value:
                curr_value = new_value
                k = 0
            else:
                # u suprotnom se restaurira vrednost resenja i prelazi se na narednu okolinu 
                restore(inverted, solution)
                k += 1
                
    return best_value, best_solution

In [14]:
number_of_users, number_of_resources, cost, fixed_cost = read_input('data/uflp_input.txt')
solution = initialize(number_of_resources, 0.25)
best_value, best_solution = reduced_VNS(number_of_users, number_of_resources, cost, fixed_cost, solution, 10, 3)

In [15]:
print('Najbolja vrednost: ', best_value)

Najbolja vrednost:  34.0


In [17]:
print('Najbolje resenje: ', best_solution)

Najbolje resenje:  [True, False, False]
